In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
# importing Natural Language Toolkit 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
data = pd.read_csv("../../../Data/consumer-reviews-of-amazon-products/Complete.csv",low_memory=False)

In [5]:
data.shape

(34660, 21)

In [6]:
dataframe =  data[['brand','categories','manufacturer','reviews.username','reviews.title','reviews.text','reviews.numHelpful',
                   'reviews.rating']]

In [7]:
dataframe.dropna(inplace=True)

In [8]:
dataSentiment = dataframe[['reviews.title','reviews.text','reviews.rating']]
dataSentiment.columns=['subject','body','rating']
dataSentiment['class'] = ['1' if x>3 else '0' for x in dataSentiment['rating']]

In [9]:
#if true will create vectorizer with stopwords
stopset = set(stopwords.words("english"))
vectorizer = TfidfVectorizer(stop_words=stopset,binary=True)

In [11]:
# Extract feature column 'Text'
dataSentiment['vector'] = vectorizer.fit_transform(dataSentiment[['body','subject']])

In [12]:
# Extract target column 'Class'
y = dataSentiment['class']

# Extract feature vector
X = dataSentiment['vector']

In [13]:
#Shuffle and split the dataset into the number of training and testing points
if True: 
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, train_size=0.80, random_state=42)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 27301 samples.
Testing set has 6826 samples.


In [14]:
# Import the models from sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import f1_score
from sklearn.model_selection import learning_curve,validation_curve
from sklearn.model_selection import KFold

objects = ('Multi-NB','AdaBoost', 'RF')

In [15]:
# function to train classifier
def train_classifier(clf, X_train, y_train):    
    clf.fit(X_train, y_train)

# function to predict features 
def predict_labels(clf, features):
    return(clf.predict(features))

In [16]:
# Initialize the three models
A = MultinomialNB(alpha=1.0,fit_prior=True)
B = AdaBoostClassifier(n_estimators=100)
C = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)

In [28]:
# loop to call function for each model
clf = [A,B,C]
pred_val = [0,0,0]

for a in range(0,3):
    train_classifier(clf[a], X_train, y_train)
    y_pred = predict_labels(clf[a],X_test)
    pred_val[a] = f1_score(y_test, y_pred) 
    print(pred_val[a])

ValueError: setting an array element with a sequence.